In [38]:
from xml.etree import cElementTree
class ET(cElementTree.TreeBuilder):
    def comment(self,data):
        self.start(cElementTree.Comment,{})
        self.data(data)
        self.end(cElementTree.Comment)
with open('/home/charles/py/ddt/iis/config.xml') as f:
    tree = cElementTree.parse(f,parser=cElementTree.XMLParser(target=ET()))

In [39]:
root = tree.getroot()

In [40]:
root.find('config/PHP').attrib['infoPath']

'http://assist256.gn.zing.vn/GetUserImg.aspx?uid={uid}'

In [41]:
root.find('config/PHP').attrib['infoPath']

'http://assist256.gn.zing.vn/GetUserImg.aspx?uid={uid}'

In [5]:
for add in root.findall('appSettings/add'):
    if add.attrib['key'] == 'AreaID':
        add.attrib['value'] = 'yes'
    if add.attrib['key'] == 'AreaName':
        add.attrib['value'] = 'yes'

In [6]:
tree.write('/home/charles/2.xml',encoding='utf-8',xml_declaration=True)

In [ ]:
root.f

In [7]:
import time

In [8]:
time.strftime('%Y-%m-%d')

'2017-05-01'

In [9]:
'_'.join(('a',time.strftime('%Y-%m-%d')))

'a_2017-05-01'

In [10]:
from urllib.parse import urlsplit

In [24]:
url

'http://assist256.gn.zing.vn/GetUserImg.aspx?uid={uid}'

In [25]:
from urllib.parse import urlparse

In [26]:
o = urlparse(url)

In [42]:
with open('/home/charles/py/ddt/iis/hosts') as fd:
    string = fd.read()

In [44]:
string.replace('quest256.gn.zing.vn','quest66.gn.zing.vn')

"# Copyright (c) 1993-1999 Microsoft Corp.\n#\n# This is a sample HOSTS file used by Microsoft TCP/IP for Windows.\n#\n# This file contains the mappings of IP addresses to host names. Each\n# entry should be kept on an individual line. The IP address should\n# be placed in the first column followed by the corresponding host name.\n# The IP address and the host name should be separated by at least one\n# space.\n#\n# Additionally, comments (such as these) may be inserted on individual\n# lines or following the machine name denoted by a '#' symbol.\n#\n# For example:\n#\n#      102.54.94.97     rhino.acme.com          # source server\n#       38.25.63.10     x.acme.com              # x client host\n\n127.0.0.1       localhost\n10.30.12.180 openapi2.me.zing.vn \n10.30.12.180 openapi2.me.zing.vn\n118.102.7.90 quest66.gn.zing.vn\n"

In [2]:
code = 'Oz/V1Yr2s74jHHbzU2XezXtqwdef8hCz8PiX3GsmbBsQqhBEXgg4RggJ9tl3FnD/aCLvnYbZtQYBy2WNpdESzU2qm36cAW3OE7RVHEAOXg+u3veKT+UAYK4HaTdR9XAJ1AuO0xrjJQqsYWoWOfqJDQztzJh2fSg3nrXdtu5yfwo='

In [16]:
from xml.etree import cElementTree
from shutil import move as mv
import time


class ET(cElementTree.TreeBuilder):
	def comment(self, data):
		self.start(cElementTree.Comment, {})
		self.data(data)
		self.end(cElementTree.Comment)


class XML:
	def __init__(self, file: str):
		backup = '_'.join((file, time.strftime('%Y-%m-%d')))
		mv(file, backup)
		with open(backup, 'r+',encoding='utf8') as f:
			self.tree = cElementTree.parse(f, parser=cElementTree.XMLParser(target=ET()))

In [18]:
g = XML(r'd:/py/ddt/db/Center.Service.exe.config_2017-05-02_2017-05-02_2017-05-02_2017-05-02')

In [ ]:
g.tree.

In [2]:
import win32com.client

In [3]:
def encrypt(key,content):
    data = win32com.client.Dispatch('CAPICOM.EncryptedData')
    data.Algorithm.KeyLength = 5
    data.Algorithm.Name = 2
    data.SetSecret(key)
    data.Content = content
    return data.Encrypt()

In [4]:
encrypt('my','hello')

com_error: (-2147221005, '无效的类字符串', None, None)